# Lesson: 4_sMRI_preproc_spatial_normalization

## Objectives
- Visualize templates and understand registration process

## Python Requirements
- [matplotlib](https://matplotlib.org/)
- [nibabel](https://nipy.org/nibabel/)
- [nilearn](https://nilearn.github.io/index.html)

In [ ]:
from nilearn import image

atlas = datasets.fetch_icbm152_2009()
atlas_img = image.concat_imgs((atlas["gm"], atlas["wm"], atlas["csf"]))
map_labels = {"Gray Matter": 1, "White Matter": 2, "Cerebrospinal Fluid": 3}
t1 = atlas["t1"]
img = nib.load(t1)
img.shape
plotting.plot_anat(img, title="plot_anat", vmax=200)